第一步：安装和导入必要包

In [ ]:
!pip install datasets
!pip install evaluate

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate

第二步：原始数据加载

In [ ]:
dataset = load_dataset("yelp_review_full")

第三步：数据处理

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

第四步：构建模型

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)
metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(output_dir="test_trainer",
                  eval_strategy="epoch",
                  per_device_train_batch_size=8,
                  per_device_eval_batch_size=8,
                  logging_steps=10,
                  )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_train_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

第五步：模型保存与加载

In [ ]:
trainer.save_model('./my_model/')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("./my_model/")

第六步：模型推理

In [ ]:
from transformers import pipeline

# 加载文本分类pipeline
classifier = pipeline("text-classification", model="./my_model/", tokenizer="google-bert/bert-base-cased", device=0)

# 输入文本进行推理
texts = ["这是第一个示例文本。", "这是第二个示例文本。"]
results = classifier(texts)

# 输出结果
for result in results:
    print(f"Label: {result['label']}, Score: {result['score']:.4f}")